In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
# Always make it pretty.
plt.style.use('ggplot')
# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import datetime

In [2]:
file = pd.read_csv('data/Collisions.csv')

In [3]:
file[file['SPEEDING'] == 'Y'].shape[0]
#Number of Accidents with speeding involved

9492

In [4]:
file[(file['SPEEDING'] == 'Y') & (file['FATALITIES'] > 0)].shape[0]

85

In [5]:
file[file['FATALITIES'] > 0].shape[0]

328

In [6]:
print(file['INCDTTM'].isnull().sum())
file['UNDERINFL'].isnull().sum()


0


25721

In [7]:
file[(file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1' )].shape[0]

9315

In [8]:
file[file['SERIOUSINJURIES'] > 0].shape[0]
file['UNDERINFL'].unique()
drunkdracc = file[((file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1'))].shape[0]

In [9]:
infl = file[((file['UNDERINFL'] == 'Y') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['FATALITIES'] > 0))].shape[0]


underinflratio = infl / file[file['FATALITIES'] > 0 ].shape[0]

noninfl = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]

noninfluratio = noninfl / file[file['FATALITIES'] > 0].shape[0]
#print(noninfluratio)
#print(underinflratio)
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Block')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Intersection')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['LIGHTCOND'] == 'Daylight')].shape[0])
print('drunk driving deaths / drunk driving accidents: ',(infl/drunkdracc))
nondrunkdrivingacccsdeath = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]
nondrunkdrivingacc = file[(file['UNDERINFL'] == 'N') | (file['UNDERINFL'] == '0')].shape[0]
print(nondrunkdrivingacc)
print('non drunk driving deaths / non drunk driving accidents: ',(nondrunkdrivingacccsdeath/nondrunkdrivingacc))
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc
shared_sample_freq = (nondrunkdrivingacccsdeath + infl) / denominator
shared_sample_variance = numerator * (shared_sample_freq * (1 - shared_sample_freq)) / denominator
difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))
difference_in_sample_proportions = (nondrunkdrivingacccsdeath/nondrunkdrivingacc)/(infl/drunkdracc)

p_value = 1 - difference_in_proportions.cdf(difference_in_sample_proportions)
print(p_value)

drunk driving deaths / drunk driving accidents:  0.00966183574879227
177568
non drunk driving deaths / non drunk driving accidents:  0.001267120201838169
0.0


In [10]:
seriousinjury = file[((file['UNDERINFL'] == 'Y') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['SERIOUSINJURIES'] > 0))].shape[0]
totalseriousinjury = file[file['SERIOUSINJURIES'] > 0].shape[0]
seriousinjurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['SERIOUSINJURIES'] > 0))].shape[0]
print(seriousinjury/totalseriousinjury)
print('serious injury driving accidents invoving drunk / drunk driving accidents: ',seriousinjury/drunkdracc)
print('serious injury driving accidents not involving drunk/ non drunk driving accidents: ',seriousinjurynondrunk/nondrunkdrivingacc)

0.14145516074450085
serious injury driving accidents invoving drunk / drunk driving accidents:  0.04487385936661299
serious injury driving accidents not involving drunk/ non drunk driving accidents:  0.014276220940710039


In [11]:
injury = file[((file['UNDERINFL'] == 'Y') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['INJURIES'] > 0))].shape[0]
injurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['INJURIES'] > 0))].shape[0]
totalinjury = file[file['INJURIES'] > 0].shape[0]
print(injury/totalinjury)
print(injury/drunkdracc)
print(injurynondrunk/nondrunkdrivingacc)

0.06567435901758172
0.41825013419216317
0.3051563344746801


In [12]:
print('car accidents involving drunk driver that injury someone',injury)
print('car accidents involving non drunk driver that injury someone',injurynondrunk)
print('car accidents involving drunk driver',drunkdracc)
print('car accidents involving sober driver',nondrunkdrivingacc)
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc

car accidents involving drunk driver that injury someone 3896
car accidents involving non drunk driver that injury someone 54186
car accidents involving drunk driver 9315
car accidents involving sober driver 177568


In [13]:

#file['ST_COLCODE'] = pd.to_numeric(file[file.ST_COLCODE.notnull()])
file['ST_COLCODE'] = pd.to_numeric(file['ST_COLCODE'],errors = 'coerce')
print(file[(file['FATALITIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['FATALITIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['SERIOUSINJURIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['SERIOUSINJURIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['INJURIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['INJURIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())

115
191
823
2050
6673
50121


In [15]:
latlong= file.iloc[0:101,0:2]
latlong['state'] = 'WA'
latlong
#trying to get all the zipcodes of data

,X,Y,state
0,-122.270834,47.539355,WA
1,-122.306332,47.606167,WA
2,-122.269611,47.516849,WA
3,-122.307505,47.612898,WA
4,-122.312430,47.708528,WA
5,-122.328079,47.604161,WA
6,-122.368598,47.670894,WA
7,-122.344507,47.687665,WA
8,-122.334266,47.733225,WA
9,-122.380504,47.672365,WA


In [87]:
file2 = file.copy()
#file2['INCDTTM'].str.contains('AM|PM',regex=True)
#test2 =file2['INCDTTM']
#test3 =test2[file2['INCDTTM'].str.contains('AM|PM',regex=True)]
#print(type(datetime.datetime.strptime('7/4/2019','%m/%d/%Y')))

In [42]:
for idx, my_date in enumerate(test2):
    try:
        file2.loc[idx,'INCDTTM'] = datetime.datetime.strptime(my_date,'%m/%d/%Y %I:%M:%S %p')
    except:
        file2.loc[idx,'INCDTTM'] = datetime.datetime.strptime(my_date,'%m/%d/%Y')

In [38]:
for i in range(212604):
    if not isinstance(file2.loc[i,'INCDTTM'], datetime.datetime):
        print(i)
        break

7674


In [ ]:
file

In [81]:
#file2.to_csv('data/CollionsWDatetime.csv',index=False)
file3 = pd.read_csv('data/CollionsWDatetime.csv')
#212604

In [82]:
#file3 = file3.dropna(subset=['INCDTTM'])
#file3.shape[0]
sum(file3['INCDTTM'].isnull())

0

In [64]:
file2.to_pickle('data/CollionsWDatetime.pkl.bz2', compression = 'bz2')

In [83]:
def datetimeconversion(row):
    try:
        return datetime.datetime.strptime(row['INCDTTM'],'%m/%d/%Y %I:%M:%S %p')
    except:
        return datetime.datetime.strptime(row['INCDTTM'],'%m/%d/%Y')

In [88]:
file2.apply(datetimeconversion, axis=1)

0        2019-07-04 00:00:00
1        2019-07-10 00:00:00
2        2019-04-19 18:21:00
3        2019-07-10 00:00:00
4        2019-04-19 16:16:00
5        2019-05-08 07:32:00
6        2019-07-02 00:00:00
7        2019-05-01 08:35:00
8        2019-05-10 16:20:00
9        2019-07-07 00:00:00
10       2019-03-07 00:25:00
11       2019-07-08 00:00:00
12       2019-07-06 00:00:00
13       2019-07-02 00:00:00
14       2019-05-15 19:26:00
15       2019-07-10 00:00:00
16       2019-04-26 16:42:00
17       2019-07-05 00:00:00
18       2019-04-26 14:45:00
19       2019-05-09 08:27:00
20       2019-07-05 00:00:00
21       2019-05-14 09:27:00
22       2019-07-10 00:00:00
23       2019-05-17 19:41:00
24       2019-07-06 00:00:00
25       2019-07-10 00:00:00
26       2019-04-24 09:20:00
27       2019-07-08 00:00:00
28       2019-05-30 13:00:00
29       2019-05-14 18:25:00
                 ...        
212574   2019-01-23 19:34:00
212575   2018-12-29 17:20:00
212576   2018-11-21 11:41:00
212577   2019-

In [326]:
file2

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.270834,47.539355,1,320868,322368,E937083,Unmatched,Block,NaN,RAINIER AVE S BETWEEN S FRONTENAC ST AND S MYR...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,N
1,-122.306332,47.606167,2,321032,322532,E938898,Unmatched,Intersection,30190.0,20TH AVE AND E JEFFERSON ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,N
2,-122.269611,47.516849,3,320892,322392,E921919,Matched,Intersection,35484.0,RENTON AVE S AND S ROXBURY ST,...,Wet,Daylight,NaN,NaN,NaN,10.0,Entering at angle,0,0,N
3,-122.307505,47.612898,4,321019,322519,E938895,Unmatched,Intersection,30145.0,19TH AVE AND E UNION ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,N
4,-122.312430,47.708528,5,320887,322387,E921700,Matched,Intersection,36506.0,15TH AVE NE AND NE NORTHGATE WAY,...,Wet,Daylight,NaN,NaN,NaN,28.0,From opposite direction - one left turn - one ...,0,0,N
5,-122.328079,47.604161,6,320881,322381,E920857,Matched,Intersection,29973.0,6TH AVE AND JAMES ST,...,Dry,Daylight,NaN,NaN,NaN,14.0,From same direction - both going straight - on...,0,0,N
6,-122.368598,47.670894,7,320866,322366,E937659,Unmatched,Intersection,25407.0,9TH AVE NW AND NW 58TH ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,N
7,-122.344507,47.687665,8,320871,322371,E920382,Matched,Intersection,24277.0,AURORA AVE N AND N 81ST ST,...,Dry,Daylight,NaN,NaN,NaN,14.0,From same direction - both going straight - on...,0,0,N
8,-122.334266,47.733225,9,321021,322521,E934514,Matched,Block,NaN,MERIDIAN AVE N BETWEEN N 143RD ST AND N 145TH ST,...,Dry,Daylight,NaN,NaN,NaN,14.0,From same direction - both going straight - on...,0,0,N
9,-122.380504,47.672365,10,320890,322390,E938202,Unmatched,Block,NaN,NW 60TH ST BETWEEN 17TH AVE NW AND 20TH AVE NW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,Y


In [150]:
test3.min()

datetime.datetime(2004, 1, 10, 1, 27)

In [151]:
url = 'https://api.darksky.net/forecast/'

In [152]:
with open('data/dark_sky.txt', 'r') as file:
    key = file.read()

In [161]:
latlong = '/47.44472,-122.31361,'
##[YYYY]-[MM]-[DD]T[HH]:[MM]:[SS]
time = '[2014]-[01]-[10]T[1]:[27]:[00]'
time2 = '1389346020'

In [162]:
finalurl= url+key+latlong+time2

In [163]:
r = requests.get(finalurl)

In [164]:
r.status_code

200

In [165]:
r.content

b'{"latitude":47.44472,"longitude":-122.31361,"timezone":"America/Los_Angeles","currently":{"time":1389346020,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":45.55,"apparentTemperature":41.78,"dewPoint":42.8,"humidity":0.9,"pressure":1012.59,"windSpeed":7.1,"windGust":15.46,"windBearing":202,"cloudCover":0.75,"uvIndex":0,"visibility":10},"hourly":{"summary":"Light rain overnight and in the evening.","icon":"rain","data":[{"time":1389340800,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":45.72,"apparentTemperature":41.82,"dewPoint":42.8,"humidity":0.89,"pressure":1011.21,"windSpeed":7.42,"windGust":14.25,"windBearing":205,"cloudCover":0.75,"uvIndex":0,"visibility":9.997},{"time":1389344400,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":45.65,"apparentTemperature":41.84,"dewPoint":42.8,"humidity":0.9,

In [200]:
dic = json.loads(r.content)
type(dic)

dict

In [226]:
url = 'https://api.darksky.net/forecast/'
with open('data/dark_sky.txt', 'r') as file:
    key = file.read()
latlong = '/47.44472,-122.31361,'
for i in range(5683): #5682 is number of days from beg to end
    time = 1389346020 + (i*24*60*60)
    strtime = str(time)
    finalurl= url+key+latlong+strtime
    r = requests.get(finalurl)
    dic = json.loads(r.content)
    Weather.insert_one(dic)
 
    

In [251]:
friends_of_twilight = testcoll.find({})
i = 0
for friend in friends_of_twilight:
    if i > 0:
        break
    pprint.pprint(friend['currently']['humidity'])
    i += 1

0.74


In [244]:
Weather.find

{'_id': ObjectId('5d781b8d4cadcc23e8d2595a'),
 'currently': {'apparentTemperature': 41.78,
  'cloudCover': 0.75,
  'dewPoint': 42.8,
  'humidity': 0.9,
  'icon': 'partly-cloudy-night',
  'precipIntensity': 0,
  'precipProbability': 0,
  'pressure': 1012.59,
  'summary': 'Mostly Cloudy',
  'temperature': 45.55,
  'time': 1389346020,
  'uvIndex': 0,
  'visibility': 10,
  'windBearing': 202,
  'windGust': 15.46,
  'windSpeed': 7.1},
 'daily': {'data': [{'apparentTemperatureHigh': 46.95,
    'apparentTemperatureHighTime': 1389394800,
    'apparentTemperatureLow': 39.39,
    'apparentTemperatureLowTime': 1389456000,
    'apparentTemperatureMax': 51.81,
    'apparentTemperatureMaxTime': 1389423600,
    'apparentTemperatureMin': 41.71,
    'apparentTemperatureMinTime': 1389348000,
    'cloudCover': 0.96,
    'dewPoint': 42.57,
    'humidity': 0.85,
    'icon': 'rain',
    'moonPhase': 0.35,
    'precipIntensity': 0.0048,
    'precipIntensityMax': 0.03,
    'precipIntensityMaxTime': 1389409200

In [237]:
datetime.datetime.utcfromtimestamp(1389340800)-datetime.timedelta(hours=8)

datetime.datetime(2014, 1, 10, 8, 0)

In [284]:
def getprecipitation(Weather):
    times = Weather.find({})
    info = dict()
    for time in times:
        for i in range(len(time['hourly']['data'])):
            t = time['hourly']['data'][i]['time']
            try: 
                rain = time['hourly']['data'][i]['precipIntensity']
            except:
                continue
            dt = datetime.datetime.utcfromtimestamp(t)-datetime.timedelta(hours=8)
            info[dt] = rain
    return info

In [300]:
rain = getprecipitation(Weather)
rainpd = pd.DataFrame.from_dict(rain, orient='index')


In [322]:
count = 0
lst = []
for i, index in zip(range(rainpd.size),rainpd.index):
    if rainpd.iloc[i,0] == 0:
        count += 1
    if rainpd.iloc[i,0] != 0:
        if count >= 36:
            lst.append(index)
        count = 0

In [336]:
lst

[Timestamp('2014-01-22 02:00:00'),
 Timestamp('2014-01-28 21:00:00'),
 Timestamp('2014-02-09 20:00:00'),
 Timestamp('2014-03-01 13:00:00'),
 Timestamp('2014-03-08 11:00:00'),
 Timestamp('2014-03-14 00:00:00'),
 Timestamp('2014-03-19 12:00:00'),
 Timestamp('2014-03-25 07:00:00'),
 Timestamp('2014-04-03 14:00:00'),
 Timestamp('2014-04-08 15:00:00'),
 Timestamp('2014-04-15 03:00:00'),
 Timestamp('2014-04-21 20:00:00'),
 Timestamp('2014-04-26 20:00:00'),
 Timestamp('2014-05-03 10:00:00'),
 Timestamp('2014-05-08 11:00:00'),
 Timestamp('2014-05-18 18:00:00'),
 Timestamp('2014-05-23 07:00:00'),
 Timestamp('2014-05-25 15:00:00'),
 Timestamp('2014-06-12 19:00:00'),
 Timestamp('2014-06-15 05:00:00'),
 Timestamp('2014-06-19 23:00:00'),
 Timestamp('2014-06-27 00:00:00'),
 Timestamp('2014-07-20 00:00:00'),
 Timestamp('2014-07-23 00:00:00'),
 Timestamp('2014-08-02 09:00:00'),
 Timestamp('2014-08-11 22:00:00'),
 Timestamp('2014-08-15 06:00:00'),
 Timestamp('2014-08-30 08:00:00'),
 Timestamp('2014-09-

In [166]:
from pymongo import MongoClient
import pprint

In [167]:
client = MongoClient('localhost', 27017)

In [168]:
db = client['WeatherDB']

In [169]:
Weather = db['Weather']

In [194]:
testcoll = db['testcoll']
PrecpIntAndProbHourly = db['PrecpIntAndProbHourly']

(1, 2)